In [ ]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components.two_wheels import TwoWheelsV3
from components import (
    CallChannel, 
    ComponentStarter, 
    LoggerComponent,
    BlueToothCarControlSPP_V2,
    start_bluetooth_server_v2b, 
    AngularSpeedControlV3,
    Picamera2V2,
    PitchAngularVelocityController, 
    get_switches
)

from data_collection.data_collection import LoggerSet
from multiprocessing import Manager
import numpy as np
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [ ]:
loggerset = LoggerSet('../log/audio_sample'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [ ]:

logger_process = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    )
)
two_wheel_process = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_process = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_process = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser_process_man = start_bluetooth_server_v2b(manager)

mic_com = ComponentStarter(
    MicrophoneComponent, 
    manager, 
    instantiator=MicrophoneComponent.entry, 
    init_kwargs = dict(
        device_indices=[9, 10]
    )
)


In [ ]:
two_wheel_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

angular_speed_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

bluetooth_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)
mic_com.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])

)


two_wheel_process.register_incoming_samples(
    angular_speed_control_process.outgoing_sample_readers
)

angular_speed_control_process.register_incoming_samples(
    bluetooth_control_process.outgoing_samples
)

bluetooth_control_process.register_incoming_samples(
    bt_ser_out
)


In [ ]:
bt_ser_process_man.start()
bluetooth_control_process.start()
logger_process.start()
two_wheel_process.start()
angular_speed_control_process.start()
mic_com.start()

In [ ]:
bt_ser_process_man.kill()
bluetooth_control_process.process_starter.kill()
#logger_process.process_starter.kill()
two_wheel_process.process_starter.kill()
angular_speed_control_process.process_starter.kill()
mic_com.process_starter.kill()

